In [2]:
%load_ext autoreload
%autoreload 2
from geolifeclef.utils import get_spark
from pyspark.sql import functions as F

spark = get_spark()
data_root = "/mnt/data/geolifeclef-2024/data"


species_edges = (
    spark.read.parquet(f"{data_root}/processed/geolsh_nn_graph/v2/species_edges")
    .where("threshold=10000")
    .cache()
)
species_edges.printSchema()
species_edges.show(10)
species_edges.count()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


24/04/07 09:29:21 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


root
 |-- srcSpeciesId: double (nullable = true)
 |-- dstSpeciesId: double (nullable = true)
 |-- n: long (nullable = true)
 |-- threshold: integer (nullable = true)



+------------+------------+---+---------+
|srcSpeciesId|dstSpeciesId|  n|threshold|
+------------+------------+---+---------+
|      3438.0|      5783.0|  7|    10000|
|       471.0|      2145.0|  9|    10000|
|      9881.0|      3918.0| 13|    10000|
|      4469.0|      8376.0|  8|    10000|
|      7536.0|      3380.0| 16|    10000|
|      8968.0|      3043.0|  1|    10000|
|      1578.0|      1226.0| 10|    10000|
|      5384.0|     10372.0| 17|    10000|
|      1687.0|     10437.0| 11|    10000|
|      1724.0|     11026.0|  1|    10000|
+------------+------------+---+---------+
only showing top 10 rows



7233877

In [4]:
from node2vec.spark import Node2VecSpark

n2v_params = {
    "num_walks": 30,
    "walk_length": 10,
    "return_param": 1.0,
    "inout_param": 1.0,
}
w2v_params = {}
g2v = Node2VecSpark(
    spark,
    n2v_params,
    w2v_params=w2v_params,
    max_out_degree=10000,
)

In [6]:
g2v.preprocess_input_graph(
    species_edges.select(
        F.col("srcSpeciesId").alias("src"),
        F.col("dstSpeciesId").alias("dst"),
        F.log1p("n").alias("weight"),
    ),
    indexed=False,
    directed=False,
)